### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,42299858,86975,79085,51530,0,0,1122,5,2025-01-09 22:00,2025-01-09 22:00,1,0,3433,4,1
34,42299678,22995,79085,3,0,0,1122,6,2025-01-10 16:29,2025-01-10 16:29,4,0,10029,3,1
35,42303392,79085,107614,51530,0,0,1123,0,2025-01-11 22:00,2025-01-11 22:00,0,4,3315,7,1
36,42084916,79085,6295,1349,7,69,1123,1,2025-01-12 21:58,2025-01-12 21:59,4,1,11308,5,1
37,42303453,79085,8323,51530,0,0,1123,2,2025-01-13 22:00,2025-01-13 22:00,11,0,3054,6,1
38,42303598,79560,79085,51530,0,0,1123,3,2025-01-14 22:00,2025-01-14 22:00,0,4,1708,3,1
39,42084928,79085,122720,1349,8,69,1123,4,2025-01-15 21:58,2025-01-15 21:59,3,0,14948,7,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,86975,27301,36343,0,15,4,1,0,DEUTSCH,10,8,9,42299858
1,79085,42656,33614,4,3,3,2,0,3-5-2aSL,8,10,10,42299858
0,22995,18926,39346,0,20,0,0,0,Optimas,9,9,10,42299678
1,79085,51038,30618,0,0,2,0,1,5-4-1a,5,7,7,42299678
0,79085,50820,31968,0,1,4,0,0,5-4-1a,5,7,6,42303392
1,107614,19146,37998,1,25,3,1,0,Her_wind_b,11,10,8,42303392
0,79085,36664,34809,7,12,3,0,0,3-4-3aQJ,9,9,11,42084916
1,6295,33293,35148,0,14,2,1,0,4132,10,10,12,42084916
0,79085,20546,42675,2,33,0,0,0,4-2-4a,10,9,10,42303453
1,8323,49594,27465,0,0,1,1,0,443 tech,8,6,4,42303453


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1504

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,47841,8,24,8,0,0,38,5,24533038008010,1349
1,6295,8,18,6,0,2,46,18,18528046006009,1349
2,23755,8,16,5,1,2,40,14,16526040005011,1349
3,78671,8,15,5,0,3,35,10,15525035005007,1349
4,79866,8,14,4,2,2,14,9,14505014004005,1349
5,89569,8,13,4,1,3,20,14,13506020004002,1349
6,122169,8,12,4,0,4,48,30,12518048004004,1349
7,124139,8,12,4,0,4,23,17,12506023004008,1349
8,79085,8,11,3,2,3,17,8,11509017003006,1349
9,48541,8,3,1,0,7,8,59,3449008001001,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()